# Browser Fingerprinting and Attack on Canvas Fingerprint Defender


In [1]:
%%writefile index.html
<!DOCTYPE html>
<html>
<head>
  <title>Canvas Fingerprint Test</title>
</head>
<body>
  <canvas id="testCanvas" width="300" height="100"></canvas>
  <script>
    const canvas = document.getElementById('testCanvas');
    const ctx = canvas.getContext('2d');
    let spoofingDetected = false;

    // Draw a blue rectangle (should be pure blue if not modified)
    ctx.fillStyle = "#007FFF";  // Pure blue (0,127,255)
    ctx.fillRect(0, 0, canvas.width, canvas.height);

    // Draw fingerprinting text on top
    ctx.font = "16px Arial";
    ctx.fillStyle = "#000";
    ctx.fillText("Fingerprint Test", 10, 50);

    // Capture the canvas output twice (to test for inconsistency)
    const snapshot1 = canvas.toDataURL();

    // Force re-read by clearing and redrawing the canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);
    ctx.fillStyle = "#007FFF";
    ctx.fillRect(0, 0, canvas.width, canvas.height);
    ctx.font = "16px Arial";
    ctx.fillStyle = "#000";
    ctx.fillText("Fingerprint Test", 10, 50);
    const snapshot2 = canvas.toDataURL();

    // If the snapshots differ, noise is present
    if (snapshot1 !== snapshot2) {
      spoofingDetected = true;
    }

    // Check a known pixel from the blue background
    const imageData = ctx.getImageData(5, 5, 1, 1).data;
    // Expected value for a pure blue background: [0,127,255,255]
    const expected = [0, 127, 255, 255];
    for (let i = 0; i < 4; i++) {
      if (imageData[i] !== expected[i]) {
        spoofingDetected = true;
        break;
      }
    }

    // Set the document title so Selenium can read it
    document.title = spoofingDetected ? "Spoofing Detected" : "No Spoofing";
    console.log(spoofingDetected ? "Canvas spoofing is ACTIVE." : "No canvas spoofing detected.");
  </script>
</body>
</html>

Overwriting index.html


In [2]:
import http.server
import socketserver
import threading

PORT = 8000

Handler = http.server.SimpleHTTPRequestHandler

def start_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print("Serving HTTP on port", PORT)
        httpd.serve_forever()

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()


Serving HTTP on port 8000


127.0.0.1 - - [05/Apr/2025 23:32:49] "GET /index.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 23:32:49] code 404, message File not found
127.0.0.1 - - [05/Apr/2025 23:32:49] "GET /favicon.ico HTTP/1.1" 404 -


In [3]:
#!/usr/bin/env python3
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time

# Set Chrome options
options = Options()
# To deactivate the extension, we can comment and uncomment this line below:
options.add_extension("/home/Semester_2/Privacy/Privacy Final Project/PluginSourceCode.crx") 

# Set path to ChromeDriver using Service
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

# Open the local HTML test file
#driver.get("file:///home/Semester_2/Privacy/Privacy Final Project/index.html")
driver.get("http://localhost:8000/index.html")

# Wait for JavaScript to execute and for you to inspect the page
time.sleep(10)  # Increased delay to 10 seconds

# Get test result from the page title set by our JS
result_text = driver.title
spoofing_detected = "Detected" in result_text
print("Canvas fingerprinting protection active:", spoofing_detected)

# Close the browser
driver.quit()


Canvas fingerprinting protection active: True


## Make it Neutral + ByPass

In [4]:
%%writefile index2.html
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Canvas Fingerprint Test with Bypass</title>
<script>
    setTimeout(() => {
      // 1) Create iframe
      var iframe = document.createElement('iframe');
      iframe.style.display = 'none';
      document.documentElement.appendChild(iframe);
  
      // 2) Grab native methods
      var nativeToDataURL = iframe.contentWindow.HTMLCanvasElement.prototype.toDataURL;
      var nativeGetImageData = iframe.contentWindow.CanvasRenderingContext2D.prototype.getImageData;
  
      // 3) Remove iframe
      document.documentElement.removeChild(iframe);
  
      // 4) Override
      HTMLCanvasElement.prototype.toDataURL = nativeToDataURL;
      CanvasRenderingContext2D.prototype.getImageData = nativeGetImageData;
  
      console.log("Re-applied native canvas methods after extension hooking.");
  
      // Now do your canvas drawing + fingerprint
      var canvas = document.getElementById('testCanvas');
      var ctx = canvas.getContext('2d');
      ctx.fillStyle = "#007FFF";
      ctx.fillRect(0, 0, 300, 100);
      ctx.font = "16px Arial";
      ctx.fillStyle = "#000";
      ctx.fillText("Fingerprint Test", 10, 50);
  
      var fingerprint = canvas.toDataURL();
      console.log("Fingerprint:", fingerprint);
      document.title = "Fingerprint: " + fingerprint.substring(0, 50);
  
    }, 5000); // Delay 2 seconds
  </script>  
</head>
<body>
  <canvas id="testCanvas" width="300" height="100"></canvas>
  <script>
    // === Fingerprinting Code (Executes AFTER Bypass) ===
    var canvas = document.getElementById('testCanvas');
    var ctx = canvas.getContext('2d');

    // Draw a blue rectangle (should be pure blue if no noise is applied)
    ctx.fillStyle = "#007FFF";  // Interpreted as rgb(0,127,255)
    ctx.fillRect(0, 0, canvas.width, canvas.height);

    // Draw fingerprinting text
    ctx.font = "16px Arial";
    ctx.fillStyle = "#000000";
    ctx.fillText("Fingerprint Test", 10, 50);

    // Generate the fingerprint using the (hopefully restored) native toDataURL method
    var fingerprint = canvas.toDataURL();
    console.log("Fingerprint:", fingerprint);
    
    // Update the document title with part of the fingerprint for verification
    document.title = "Fingerprint: " + fingerprint.substring(0, 50);
  </script>
</body>
</html>

Overwriting index2.html


In [5]:
import http.server
import socketserver
import threading

PORT = 8050

Handler = http.server.SimpleHTTPRequestHandler

def start_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print("Serving HTTP on port", PORT)
        httpd.serve_forever()

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()


Serving HTTP on port 8050


127.0.0.1 - - [05/Apr/2025 23:33:42] "GET /index2.html HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 23:33:42] code 404, message File not found
127.0.0.1 - - [05/Apr/2025 23:33:42] "GET /favicon.ico HTTP/1.1" 404 -


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import os

# Configure Chrome Options
options = Options()
# To deactivate the extension, we can comment and uncomment this line below:
extension_path = "/home/Semester_2/Privacy/Privacy Final Project/PluginSourceCode.crx"
if not os.path.exists(extension_path):
    raise FileNotFoundError(f"Extension not found at {extension_path}")
options.add_extension(extension_path)

# Set Up ChromeDriver Service
chromedriver_path = "/usr/bin/chromedriver"
if not os.path.exists(chromedriver_path):
    raise FileNotFoundError(f"ChromeDriver not found at {chromedriver_path}")
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# Load the Test Page from our HTTP Server
driver.get("http://localhost:8050/index2.html")
time.sleep(10)  # Wait for page to load and JS to execute

# Capture and print the result
page_title = driver.title
print("Page title:", page_title)

if page_title.startswith("Fingerprint:"):
    print("Bypass successful: Native canvas fingerprint recovered.")
else:
    print("Bypass failed or not detected.")
driver.quit()


Page title: Fingerprint: data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAASwA
Bypass successful: Native canvas fingerprint recovered.
